In [37]:
from sklearn.linear_model import Ridge
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
data=pd.read_excel(r"C:\Users\User\Desktop\Machine Learning Project\Biodiesel-Yield-Using-ANN-main\Xantham_gum_data.xlsx")
data

,X1,X2,X3,Observed
0,1,2,7.5,7.22
1,3,2,7.5,7.52
2,1,6,7.5,4.58
3,3,6,7.5,6.45
4,1,4,0.0,4.47
5,3,4,0.0,5.67
6,1,4,15.0,6.46
7,3,4,15.0,7.76
8,2,2,0.0,6.65
9,2,6,0.0,4.52


In [39]:
def get_preds(y_test, y_preds):
    y_test=pd.DataFrame(y_test)
    y_test.rename(columns={0:'Actual'}, inplace=True)
    y_preds=pd.DataFrame(y_preds)
    y_preds.rename(columns={0:'Predicted'}, inplace=True)
    predictions=pd.concat([y_test, y_preds], axis=1)
    return predictions    

In [40]:
target="Observed"
X=data.drop(columns=[target])
y=data[target]

In [41]:
from sklearn.model_selection import train_test_split, cross_val_score,KFold, GridSearchCV
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from math import sqrt
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error

In [42]:
X1=X.values
y1=y.values.reshape(-1,1)
X_train, X_test, y_train, y_test= train_test_split(X1, y1, test_size=0.3, random_state=100)
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)
x_df=sc.transform(X1)
sc_y=StandardScaler()
y_train=sc_y.fit_transform(y_train)
y_test=sc_y.transform(y_test)
y_df=sc_y.transform(y1)

In [43]:
from sklearn.kernel_ridge import KernelRidge

In [44]:
kr=GridSearchCV(
    KernelRidge(),
    param_grid={"kernel":["rbf","poly"],
                "alpha" : [1e-1,1e10,0.1,1e-2,1e-3], 
                "gamma":np.logspace(-3,2,5)},cv=3
)

In [45]:
import time
start_time=time.time()
kr.fit(X_train,y_train)
elapsed_time=time.time()-start_time
print(f'executed in {elapsed_time} seconds or {elapsed_time*1000} milliseconds')

executed in 0.3630859851837158 seconds or 363.0859851837158 milliseconds


In [46]:
kr.best_estimator_

KernelRidge(alpha=0.001, gamma=0.01778279410038923, kernel='poly')

In [47]:
kr.best_params_

{'alpha': 0.001, 'gamma': 0.01778279410038923, 'kernel': 'poly'}

In [48]:
pred_test=kr.predict(X_test)

In [49]:
r2_score(y_test,pred_test)

0.9292410342951153

In [50]:
y_test1 = sc_y.inverse_transform(y_test)
pred1 = sc_y.inverse_transform(pred_test.reshape(-1,1))
krr_predictions_test=get_preds(y_test1, pred1)
krr_predictions_test

,Actual,Predicted
0,6.28,6.110268
1,4.52,4.221241
2,6.93,6.653263
3,7.52,7.809661
4,5.81,6.110268
5,6.46,6.542372


In [51]:
corr_test=krr_predictions_test["Actual"].corr(krr_predictions_test["Predicted"])
corr_test

0.9769172788293619

In [52]:
print("TEST Mean absolute error :", mean_absolute_error(y_test1,pred1))
print("TEST Mean squared error :",mean_squared_error(y_test1,pred1))
print("TEST Root Mean squared error :",sqrt(mean_squared_error(y_test1,pred1)))
print("TEST R2 Score :",r2_score(y_test1,pred1))

TEST Mean absolute error : 0.23625490012555414
TEST Mean squared error : 0.06258316032305027
TEST Root Mean squared error : 0.25016626535776215
TEST R2 Score : 0.9292410342951154


In [53]:
pred_train=kr.predict(X_train)

In [54]:
r2_score(y_train,pred_train)

0.9938209780360656

In [55]:
y_train1 = sc_y.inverse_transform(y_train)
pred_train1 = sc_y.inverse_transform(pred_train.reshape(-1,1))
krr_predictions_train=get_preds(y_train1, pred_train1)
krr_predictions_train

,Actual,Predicted
0,6.22,6.110268
1,5.67,5.688543
2,4.58,4.576302
3,4.47,4.484201
4,8.31,8.304427
5,7.22,7.207691
6,6.21,6.110268
7,7.76,7.764298
8,6.45,6.419432
9,5.86,6.110268


In [56]:
corr_train=krr_predictions_train["Actual"].corr(krr_predictions_train["Predicted"])
corr_train

0.9969170657020869

In [57]:
print("TRAIN Mean absolute error :", mean_absolute_error(y_train1,pred_train1))
print("TRAIN Mean squared error :",mean_squared_error(y_train1,pred_train1))
print("TRAIN Root Mean squared error :",sqrt(mean_squared_error(y_train1,pred_train1)))
print("TRAIN R2 Score :",r2_score(y_train1,pred_train1))

TRAIN Mean absolute error : 0.052220286230678266
TRAIN Mean squared error : 0.007906053496224356
TRAIN Root Mean squared error : 0.08891599122893674
TRAIN R2 Score : 0.9938209780360656


In [58]:
pred_df=kr.predict(x_df)

In [59]:
r2_score(y_df,pred_df)

0.9761533266297474

In [60]:
y_df1 = sc_y.inverse_transform(y_df)
pred_df1 = sc_y.inverse_transform(pred_df.reshape(-1,1))
krr_predictions_df=get_preds(y_df1, pred_df1)
krr_predictions_df

,Actual,Predicted
0,7.22,7.207691
1,7.52,7.809661
2,4.58,4.576302
3,6.45,6.419432
4,4.47,4.484201
5,5.67,5.688543
6,6.46,6.542372
7,7.76,7.764298
8,6.65,6.624499
9,4.52,4.221241


In [61]:
print("DATA Mean absolute error :", mean_absolute_error(y_df1,pred_df1))
print("DATA Mean squared error :",mean_squared_error(y_df1,pred_df1))
print("DATA Root Mean squared error :",sqrt(mean_squared_error(y_df1,pred_df1)))
print("DATA R2 Score :",r2_score(y_df1,pred_df1))

DATA Mean absolute error : 0.11717367937004622
DATA Mean squared error : 0.027203855905692326
DATA Root Mean squared error : 0.16493591454165563
DATA R2 Score : 0.9761533266297474


In [62]:
kr_rbf=GridSearchCV(
    KernelRidge(kernel="rbf"),
    param_grid={
                "alpha" : [1e-1,1e10,0.1,1e-2,1e-3], 
                "gamma":np.logspace(-3,2,5)},cv=3
)

In [63]:
kr_rbf.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=KernelRidge(kernel='rbf'),
             param_grid={'alpha': [0.1, 10000000000.0, 0.1, 0.01, 0.001],
                         'gamma': array([1.00000000e-03, 1.77827941e-02, 3.16227766e-01, 5.62341325e+00,
       1.00000000e+02])})

In [64]:
pred_rbf_train=kr_rbf.predict(X_train)
pred_rbf_test=kr_rbf.predict(X_test)
print("TRAIN RBF Mean absolute error :", mean_absolute_error(y_train,pred_rbf_train))
print("TRAIN RBF Mean squared error :",mean_squared_error(y_train,pred_rbf_train))
print("TRAIN RBF Root Mean squared error :",sqrt(mean_squared_error(y_train,pred_rbf_train)))
print("TRAIN RBF R2 Score :",r2_score(y_train,pred_rbf_train))
print("\n")
print("TEST RBF Mean absolute error :", mean_absolute_error(y_test,pred_rbf_test))
print("TEST RBF Mean squared error :",mean_squared_error(y_test,pred_rbf_test))
print("TEST RBF Root Mean squared error :",sqrt(mean_squared_error(y_test,pred_rbf_test)))
print("TEST RBF R2 Score :",r2_score(y_test,pred_rbf_test))

TRAIN RBF Mean absolute error : 0.05110678182265229
TRAIN RBF Mean squared error : 0.0064856507542352506
TRAIN RBF Root Mean squared error : 0.0805335380710127
TRAIN RBF R2 Score : 0.9935143492457648


TEST RBF Mean absolute error : 0.2179852237042661
TEST RBF Mean squared error : 0.05333310054741599
TEST RBF Root Mean squared error : 0.2309396036790052
TEST RBF R2 Score : 0.9228455769825398


In [65]:
kr_lin=GridSearchCV(
    KernelRidge(kernel="linear"),
    param_grid={
                "alpha" : [1e-1,1e10,0.1,1e-2,1e-3], 
                "gamma":np.logspace(-3,2,5)},cv=3
)

In [66]:
kr_lin.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=KernelRidge(),
             param_grid={'alpha': [0.1, 10000000000.0, 0.1, 0.01, 0.001],
                         'gamma': array([1.00000000e-03, 1.77827941e-02, 3.16227766e-01, 5.62341325e+00,
       1.00000000e+02])})

In [67]:
pred_lin_train=kr_lin.predict(X_train)
pred_lin_test=kr_lin.predict(X_test)
print("TRAIN LIN Mean absolute error :", mean_absolute_error(y_train,pred_lin_train))
print("TRAIN LIN Mean squared error :",mean_squared_error(y_train,pred_lin_train))
print("TRAIN LIN Root Mean squared error :",sqrt(mean_squared_error(y_train,pred_lin_train)))
print("TRAIN LIN R2 Score :",r2_score(y_train,pred_lin_train))
print("\n")
print("TEST LIN Mean absolute error :", mean_absolute_error(y_test,pred_lin_test))
print("TEST LIN Mean squared error :",mean_squared_error(y_test,pred_lin_test))
print("TEST LIN Root Mean squared error :",sqrt(mean_squared_error(y_test,pred_lin_test)))
print("TEST LIN R2 Score :",r2_score(y_test,pred_lin_test))

TRAIN LIN Mean absolute error : 0.21240405417046668
TRAIN LIN Mean squared error : 0.0711892587074692
TRAIN LIN Root Mean squared error : 0.26681315317553067
TRAIN LIN R2 Score : 0.9288107412925308


TEST LIN Mean absolute error : 0.3261399785683306
TEST LIN Mean squared error : 0.1464364053707173
TEST LIN Root Mean squared error : 0.3826700999173012
TEST LIN R2 Score : 0.7881575185173437
